In [ ]:
import numpy as np
import pandas as pd
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%store -r dat
dat

## feature engineering 

In [ ]:
##feature engineering
mov=10

#moving average 
temp1 = dat.loc[:,'V1':'V15'].apply(lambda  x: x.rolling(window=mov).mean(),axis=0)
temp1.columns = 'mov avg' + temp1.columns

#moving average max
temp2 = dat.loc[:,'V1':'V15'].apply(lambda  x: x.rolling(window=mov).max(),axis=0)
temp2.columns = 'mov max' + temp2.columns

#moving average min
temp3 = dat.loc[:,'V1':'V15'].apply(lambda  x: x.rolling(window=mov).min(),axis=0)
temp3.columns = 'mov min' + temp3.columns


mov_data = pd.concat([temp1,temp2,temp3],axis=1)[9:]


In [ ]:
#add previous logs
temp1 = dat.loc[:,'V1':'V15'].shift(7)
temp1.columns = 'lag 7 ' + temp1.columns
temp2 = dat.loc[:,'V1':'V15'].shift(8)
temp2.columns = 'lag 8 ' + temp2.columns
temp3 = dat.loc[:,'V1':'V15'].shift(9)
temp3.columns = 'lag 9 ' + temp3.columns
prv_data = pd.concat([temp1,temp2,temp3],axis=1)[9:]


In [ ]:
#one hot encoding
list_one_hot = ['year','month','day','hour']
for i in list_one_hot:
    # Get one hot encoding of columns B
    one_hot = pd.get_dummies(dat[i])
    # Drop column B as it is now encoded
    dat = dat.drop(i,axis = 1)
    # Join the encoded df
    dat = dat.join(one_hot)
  
  #delete time stamp
del dat['V0']


In [140]:
a = dat_new.isnull().sum()
dat_new

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,mov minV6,mov minV7,mov minV8,mov minV9,mov minV10,mov minV11,mov minV12,mov minV13,mov minV14,mov minV15
9,-3.716719,500.059239,133.137748,-4.233095,45.753789,1.625762,1.595271,0.168875,0.976544,1.795665,...,1.395419,1.143861,-0.091569,0.763201,1.241215,1307.76,-0.597609,-1.338655,-1.037515,-2.769631
10,-3.598363,500.426342,132.936482,-4.388809,45.234997,1.758288,1.530670,0.110573,0.874004,1.683155,...,1.395419,1.143861,-0.091569,0.763201,1.241215,1307.76,-0.597609,-1.909953,-1.037515,-3.379918
11,-3.312339,500.265295,132.961460,-4.294721,44.805314,1.856280,1.777368,0.229766,1.106410,1.615100,...,1.395419,1.143861,-0.091569,0.763201,1.241215,1306.98,-1.834577,-1.909953,-2.622249,-3.379918
12,-2.982333,499.841291,133.738494,-3.724466,44.333125,1.888860,1.790434,0.199584,0.914527,1.586328,...,1.395419,1.143861,-0.091569,0.763201,1.241215,1301.73,-1.834577,-1.909953,-2.622249,-3.379918
13,-2.899299,499.893277,133.457668,-3.797008,44.483366,1.976567,1.853153,0.242760,1.135723,1.805624,...,1.421063,1.143861,-0.091569,0.763201,1.293750,1301.67,-1.834577,-1.909953,-2.622249,-3.379918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70086,-0.404982,501.040783,135.916502,-2.597715,36.011214,2.783641,1.966605,0.775285,1.138488,1.917188,...,2.016800,1.710144,0.366921,0.982746,1.685126,1254.09,-1.237687,-0.934393,-2.435449,-1.301229
70087,-0.720429,500.808688,135.873692,-3.444217,33.113073,2.818106,1.813547,0.793354,1.143923,1.967996,...,2.352713,1.710144,0.659672,0.982746,1.831553,1254.09,-1.237687,-0.934393,-2.435449,-1.367090
70088,-1.371468,500.492515,135.174733,-3.456158,36.551115,2.390426,1.920235,0.669662,1.162564,1.831300,...,2.390426,1.813547,0.669662,0.982746,1.831300,1254.09,-0.976457,-0.934393,-1.916066,-1.367090
70089,-1.402805,500.222985,135.031809,-3.587148,36.195800,2.467938,1.787150,0.808476,1.087828,1.629261,...,2.390426,1.787150,0.669662,1.087828,1.629261,1254.09,-0.974376,-0.934393,-1.860779,-1.367090


In [139]:
#Connect of all new databases
dat_new = pd.concat([dat[9:],prv_data,mov_data],axis=1)
#Sanity test
print(dat_new.shape)
print(dat.shape,prv_data.shape,mov_data.shape)
print('******')
#Divide data to train\test, x\y, 7 hours ahead
#x/y
y_dat = dat_new.loc[:,'V1']
x_dat = dat_new.loc[:, dat_new.columns != 'V1']

#shift 7 hours ahead  
y_dat = y_dat[7:]
x_dat = x_dat[:-7]

#train\test
x_train, x_test, y_train, y_test = train_test_split(x_dat, y_dat, test_size=0.2)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)


(70082, 180)
(70091, 90) (70082, 45) (70082, 45)
******
(56060, 179) (14015, 179)
(56060,) (14015,)


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphidata['lag_1'] = data['Count'].shift(1)cs.tsaplots import plot_pacf
plot_acf(dat['V1'], lags=50)
plot_pacf(dat['V1'], lags=50)

## model fit

In [ ]:
a = x_test.isnull().sum()

In [154]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
import xgboost as xgb

#fff

validation_size = 0.20
seed = 7

# Spot Check Algorithms
models = []
models.append(('LR', LinearRegression()))
models.append(('EN', ElasticNet(alpha=1.0, l1_ratio=0.5)))
models.append(('XG',xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)))

data_dmatrix = xgb.DMatrix(data=x_train,label=y_train)

# evaluate each model in turn
results = []
names = []
rmses = []
for name, model in models:
    model.fit(x_train, y_train)
    y_predict = model.predict(x_test)
    results.append(y_predict)
    names.append(name)
    rmse = sqrt(mean_squared_error(y_test, y_predict))
    rmses.append(rmse)
    msg = "%s: %f" % (name,rmse)
    print(msg)



LR: 1.176029
EN: 1.638566
XG: 2.710509


In [150]:
data_dmatrix = xgb.DMatrix(data=x_train,label=y_train)
model = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
model.fit(x_train,y_train)
y_predict = model.predict(x_test)
sqrt(mean_squared_error(y_test, y_predict))

[20:41:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


2.710509331708676

In [ ]:
#LinearRegression
from sklearn.linear_model import LinearRegression

x_train = x_train.apply(pd.to_numeric, errors='coerce')
x_test = x_test.apply(pd.to_numeric, errors='coerce')

model = LinearRegression()
model.fit(x_train, y_train)

y_predict_reg = model.predict(x_test)



In [ ]:
#Elastic Net
from sklearn.linear_model import ElasticNet
model = ElasticNet(alpha=1.0, l1_ratio=0.5)
model.fit(x_train, y_train)
y_predict_elas = model.predict(x_test)


In [ ]:
#random forest
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
model.fit(x_train, y_train)
y_predict_RF = model.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, y_predict_elas))
rms